# Models 

- eval(train, test, 16, 64*4, 2, device, 0.001)
    - batch_size=100, num_epochs=1000, print_point=5
- eval(train, test, 16, 64*5, 2, device, 0.001) 2

In [1]:
import torch
import torch.nn as nn
from utils.dataset import CSVDataset, CSVDatasetGroupSplit
from torch.utils.data import Dataset, DataLoader
import math
import numpy as np
import pandas as pd

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size*2)
        self.l3 = nn.Linear(hidden_size*2, hidden_size*4)
        self.l4 = nn.Linear(hidden_size*4, num_classes)
        self.relu = nn.ReLU()

    
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        #x = self.relu(self.l3(x))
        #x = self.relu(self.l4(x))
        # no activation and no softmax at the end
        return x
    
def eval(train, test, input_size, hidden_size, num_classes, device, learning_rate, batch_size=100, num_epochs=1000, print_point=10):
    # Set Params
    total_samples = len(train)
    n_iterations = math.ceil(total_samples/batch_size)

    # Data loader
    loader = DataLoader(dataset=train,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=1)

    model = NeuralNet(input_size, hidden_size, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(loader):
              
            #inputs = inputs.reshape(-1, input_size)
            inputs = inputs.to(device)
            labels = labels.squeeze().type(torch.LongTensor).to(device)

            #print(labels.shape)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % print_point == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_iterations}], Loss: {loss.item():.4f}')
                #print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')

    PATH = './models/asd_16_ffn_4.pth'
    torch.save(model.state_dict(), PATH)

    # Data loader Test
    loader = DataLoader(dataset=test,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=1) 
    
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.squeeze().type(torch.LongTensor).to(device)
            outputs = model(inputs)     
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            acc = 100.0 * n_correct / n_samples
            print(f'Accuracy of the network on the 10000 test images: {acc} %')



# Load Datasets Train
file_path = './data/dataframes/all_bands_test1.csv'
df = pd.read_csv(file_path, dtype=np.float32)

df_theta_alpha = df[['P_T','P_A', "F_T", 'F_A', 'P_B','P_D', "F_B", 'F_D']] # x
label = df['class'] # y
group = df['group'] # Group

print(group.shape, df_theta_alpha.shape), 
#train, test = CSVDatasetGroupSplit(df_theta_alpha, df['group']).getSplit()
train, test = CSVDatasetGroupSplit(df_theta_alpha, label, group).getSplit()


# 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# train, test, input_size, hidden_size, num_classes, device, learning_rate
# eval(train, test, 16, 64*4, 2, device, 0.001)
eval(train, test, 8, 64*1, 2, device, 0.001)

/home/csc/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(4595,) (4595, 8)
train, test (3604, 8) (991, 8)
Epoch [1/1000], Step [10/37], Loss: 10.1562
Epoch [1/1000], Step [20/37], Loss: 7.7205
Epoch [1/1000], Step [30/37], Loss: 4.7291
Epoch [2/1000], Step [10/37], Loss: 3.5548
Epoch [2/1000], Step [20/37], Loss: 3.5300
Epoch [2/1000], Step [30/37], Loss: 3.5034
Epoch [3/1000], Step [10/37], Loss: 3.4543
Epoch [3/1000], Step [20/37], Loss: 3.8078
Epoch [3/1000], Step [30/37], Loss: 3.3218
Epoch [4/1000], Step [10/37], Loss: 3.5937
Epoch [4/1000], Step [20/37], Loss: 3.7470
Epoch [4/1000], Step [30/37], Loss: 3.4017
Epoch [5/1000], Step [10/37], Loss: 3.8588
Epoch [5/1000], Step [20/37], Loss: 3.6190
Epoch [5/1000], Step [30/37], Loss: 3.6492
Epoch [6/1000], Step [10/37], Loss: 3.6220
Epoch [6/1000], Step [20/37], Loss: 2.8555
Epoch [6/1000], Step [30/37], Loss: 3.3839
Epoch [7/1000], Step [10/37], Loss: 3.6175
Epoch [7/1000], Step [20/37], Loss: 3.5503
Epoch [7/1000], Step [30/37], Loss: 3.3339
Epoch [8/1000], Step [10/37], Loss: 3.8084
Epoc